<a href="https://colab.research.google.com/github/akheel8743/Brolly1/blob/main/Shopify_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import files

uploaded = files.upload()


Saving sample_customer_reviews_raw.csv to sample_customer_reviews_raw.csv


In [10]:
df = pd.read_csv("raw_reviews.csv")


In [11]:
import pandas as pd
import numpy as np


In [12]:
# Load the CSV
df = pd.read_csv("raw_reviews.csv")
print("✅ Loaded raw data:", df.shape)

# Standardize timestamps
df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')

# Normalize product name/category
df['Product Name'] = df['Product Name'].astype(str).str.lower().str.strip()
df['Product Category'] = df['Product Category'].astype(str).str.lower().str.strip()

# Filter out short/blank reviews
df['Review Content'] = df['Review Content'].astype(str)
df = df[df['Review Content'].str.len() >= 10]

# Handle rating
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')
df = df[df['Rating'].between(1, 5, inclusive='both')]
df['Rating'].fillna(df['Rating'].median(), inplace=True)

# Flag missing values
df['Missing_Order_Value'] = df['Order Value'].isnull()
df['Missing_Fulfillment_Status'] = df['Fulfillment Status'].isnull()

print("✅ Cleaned dataset:", df.shape)


✅ Loaded raw data: (100, 10)
✅ Cleaned dataset: (73, 12)


<ipython-input-12-e9d4fff489a8>:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Rating'].fillna(df['Rating'].median(), inplace=True)


In [13]:
df.to_csv("cleaned_reviews.csv", index=False)
files.download("cleaned_reviews.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
q1 = df[(df['Rating'] == 1) & (df['Shipping Country'].str.lower() == 'canada')]
q1_result = q1['Product Category'].value_counts().head(10)
print("🇨🇦 Categories with most 1-star reviews in Canada:\n", q1_result)


🇨🇦 Categories with most 1-star reviews in Canada:
 Product Category
prom dresses    1
nan             1
promdrsses      1
Name: count, dtype: int64


In [15]:
df['Order Value'] = pd.to_numeric(df['Order Value'], errors='coerce')
correlation = df[['Order Value', 'Rating']].dropna().corr().iloc[0, 1]
print("📉 Correlation between Order Value and Rating:", correlation)


📉 Correlation between Order Value and Rating: -0.03104884688289284


In [16]:
from sklearn.feature_extraction.text import CountVectorizer

def get_top_phrases(data, n=5):
    vectorizer = CountVectorizer(ngram_range=(1, 2), stop_words='english', max_features=2000)
    matrix = vectorizer.fit_transform(data)
    sums = matrix.sum(axis=0)
    words_freq = [(word, sums[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
    sorted_words = sorted(words_freq, key=lambda x: x[1], reverse=True)
    return sorted_words[:n]

# Complaints (ratings ≤ 2)
complaints = get_top_phrases(df[df['Rating'] <= 2]['Review Content'])
print("❗ Top Complaints:", complaints)

# Compliments (ratings ≥ 4)
compliments = get_top_phrases(df[df['Rating'] >= 4]['Review Content'])
print("🎉 Top Compliments:", compliments)


❗ Top Complaints: [('sing', np.int64(3)), ('right', np.int64(3)), ('wait', np.int64(3)), ('necessary', np.int64(3)), ('prove', np.int64(3))]
🎉 Top Compliments: [('wonder', np.int64(3)), ('simple', np.int64(2)), ('real', np.int64(2)), ('author', np.int64(2)), ('social', np.int64(2))]


In [17]:
negatives = df[df['Rating'] <= 2]
fulfillment_impact = negatives['Fulfillment Status'].value_counts()
print("📦 Fulfillment statuses with most negative reviews:\n", fulfillment_impact)


📦 Fulfillment statuses with most negative reviews:
 Fulfillment Status
Delaye       6
Returned     6
Fufilled     5
Cancelled    4
Delayed      3
Fulfilled    2
Name: count, dtype: int64
